In [7]:
import sys
sys.path.append('../')

from autogpt.config import Config
import json
import requests
import yaml

from duckduckgo_search import ddg


CFG = Config()

## Google API
https://developers.naver.com/docs/serviceapi/search/movie/movie.md#%EC%98%81%ED%99%94

In [4]:
import json

In [27]:
def google_search_place(CFG, place_name: str, num_results: int = 10) -> str:
    api_key = CFG.google_api_key
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place_name}&inputtype=textquery&fields=place_id,name&key={api_key}"
    result = requests.get(url)
    json_obj = result.json()

    # if status is not OK, return failed to find place
    if json_obj['status'] != 'OK':
        return {'status': 'failed to find place'}

    candidates = []
    for candidate in json_obj['candidates'][:num_results]:
        place_id = candidate['place_id']  # Get the place ID of the first result
        place_details = get_place_details(place_id)

        # if place details include 'status' key, return failed to find place
        if 'status' in place_details:
            continue

        place_details['address_components'] = [comp['short_name'] for comp in place_details['address_components'][1:-2]]
        place_details['location'] = place_details['geometry']['location']
        candidate.update(place_details)
        del candidate['place_id'], candidate['geometry']
        candidates.append(candidate)

    return json.dumps(candidates, ensure_ascii=False)

def get_place_details(place_id):
    api_key = CFG.google_api_key
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=geometry/location,rating,types,address_components&key={api_key}"
    result = requests.get(url)
    json_obj = result.json()

    # if status is not OK, return failed to find place
    if json_obj['status'] != 'OK':
        return {'status': 'failed to find place'}

    return json_obj['result']

def google_search_nearby_places(latitude, longitude, radius: int=1000, num_results: int = 10) -> str:
    api_key = CFG.google_api_key
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&key={api_key}"
    result = requests.get(url)
    json_obj = result.json()
    
    candidates = []
    for candidate in json_obj['results'][:num_results]:
        place_id = candidate['place_id']  # Get the place ID of the first result
        place_details = get_place_details(place_id)
        candidate = {k:v for k, v in candidate.items() if k in ['name', 'place_id']}
        place_details['address_components'] = [comp['short_name'] for comp in place_details['address_components'][1:-2]]
        place_details['location'] = place_details['geometry']['location']
        candidate.update(place_details)
        del candidate['place_id'], candidate['geometry']
        candidates.append(candidate)

    return json.dumps(candidates, ensure_ascii=False)


In [ ]:
google_search_place('Eiffel Tower')

'[{"name": "Torre Eiffel", "address_components": ["Av. Anatole France", "Paris", "Département de Paris", "IDF"], "rating": 4.6, "types": ["tourist_attraction", "point_of_interest", "establishment"], "location": {"lat": 48.85837009999999, "lng": 2.2944813}}, {"name": "Bureau de Gustave Eiffel", "address_components": ["5", "Av. Anatole France", "Paris", "Département de Paris", "IDF"], "rating": 4.5, "types": ["tourist_attraction", "point_of_interest", "establishment"], "location": {"lat": 48.85805199999999, "lng": 2.2948624}}]'

In [28]:
result = google_search_place(CFG, '파도살롱')

In [30]:
json.loads(result)

[{'name': '파도살롱',
  'address_components': ['Gyeonggang-ro', 'Gangneung-si', 'Gangwon-do'],
  'rating': 4.3,
  'types': ['real_estate_agency', 'point_of_interest', 'establishment'],
  'location': {'lat': 37.74995759999999, 'lng': 128.8906032}}]

In [35]:
# get latitude and longitude of a place
location = json.loads(result)[0]['location']

# max 7 decimal places
lat, lng = round(location['lat'], 7), round(location['lng'], 7)
location_str = f'{lat},{lng}'
location_str

'37.7499576,128.8906032'

In [11]:
place_name = '파도살롱'
api_key = CFG.google_api_key
url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place_name}&inputtype=textquery&fields=place_id,name&key={api_key}"
result = requests.get(url)
json_obj = result.json()

In [12]:
json_obj

{'candidates': [{'name': '파도살롱', 'place_id': 'ChIJ8zD9JOHvYTURIJJHGlYJbJs'}],
 'status': 'OK'}

In [13]:
place_id = json_obj['candidates'][0]['place_id']

In [14]:
url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=geometry/location,rating,types,address_components&key={api_key}"

result = requests.get(url)
json_obj = result.json()

In [17]:
json_obj

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '2023',
    'short_name': '2023',
    'types': ['premise']},
   {'long_name': 'Gyeonggang-ro',
    'short_name': 'Gyeonggang-ro',
    'types': ['sublocality_level_4', 'sublocality', 'political']},
   {'long_name': 'Gangneung-si',
    'short_name': 'Gangneung-si',
    'types': ['locality', 'political']},
   {'long_name': 'Gangwon-do',
    'short_name': 'Gangwon-do',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'South Korea',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '25535', 'short_name': '25535', 'types': ['postal_code']}],
  'geometry': {'location': {'lat': 37.74995759999999, 'lng': 128.8906032}},
  'rating': 4.3,
  'types': ['real_estate_agency', 'point_of_interest', 'establishment']},
 'status': 'OK'}

In [19]:
latitude = json_obj['result']['geometry']['location']['lat']
longitude = json_obj['result']['geometry']['location']['lng']
radius = 1000

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&key={api_key}"
result = requests.get(url)
json_obj = result.json()

In [20]:
json_obj

{'html_attributions': [],
 'next_page_token': 'AZose0luhcdAYwsV2oB6HOak9HxFmBS1aEpwjxVp6gOA5e22UKyXkJqddE-FvaxDdwIVAnJ2fa9bQazHiYs9kRIay3njAaQWz2xEhf_7c4mYZ8Ow-9w4FlAW7Q6f1-gsp3uy6THefJDAjkSDTxOxUGu6u9j1tSHP1OrVCPPd6hkEBDNilk1V827KcWbXTg13F-xqPmBQZupAbrIu--vg1BY0oK_6s3llEati8falUw3DdnDbzhOQXT7X7BgX5kxaRfLN_sFd_7tbhW7umFaeRY34L_at2CiNljaxtToZZfnUGmTjFP-fzmopoRRll1a0pIESkBOXzFi0J2_1pJpN2kMbtKWMEgjx8V_ZfpT12ayob5O1deKsoEg6du4Py7oj0LvpM79sTibqswabg1jVPWjP1AG51PBpilsJAih1iCdX-J2N',
 'results': [{'geometry': {'location': {'lat': 37.7091295, 'lng': 128.8324462},
    'viewport': {'northeast': {'lat': 37.91736287370577,
      'lng': 129.0744516300895},
     'southwest': {'lat': 37.50329535018169, 'lng': 128.5803458145292}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Gangneung-si',
   'photos': [{'height'